In [6]:
import zipfile
import os
import random
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import (
    confusion_matrix, roc_auc_score, accuracy_score, f1_score,
    precision_recall_curve, auc, make_scorer
)
from scipy.stats import loguniform, uniform

# Load data
!git clone https://github.com/jveenland/tm10007_ml.git

with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')

data = pd.read_csv('/content/tm10007_ml/ecg/ecg_data.csv', index_col=0)

# Split features and labels
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

outer = range(10)  # Adjust as needed

best_params_dict = {}
best_model_dict = {}

# Initialize lists for PR curve data
precision_list = []
recall_list = []
auc_list = []

for outer_rand in tqdm(outer, desc='Outer Loop'):
    # Outer split
    X_train_outer, X_test_outer, y_train_outer, y_test_outer = train_test_split(
        x, y, test_size=0.2, stratify=y, shuffle=True, random_state=outer_rand
    )

    # Scaling
    scaler = MinMaxScaler()
    X_train_outer = scaler.fit_transform(X_train_outer)
    X_test_outer = scaler.transform(X_test_outer)

    # PCA
    pca = PCA(n_components=0.99, random_state=42)
    X_train_outer = pca.fit_transform(X_train_outer)
    X_test_outer = pca.transform(X_test_outer)

    # Model pipeline
    pipeline = Pipeline([
        ('svc', SVC(kernel='rbf', probability=True))
    ])

    # Hyperparameter space
    param_grid_svc = {
        'svc__C': loguniform(1e-2, 1e2),  # was 0.01 to 1000
        'svc__gamma': loguniform(1e-4, 1),  # was 0.0001 to 100
        'svc__class_weight': ['balanced', None],
        'svc__shrinking': [True, False],
        'svc__tol': uniform(1e-4, 1e-3),  # was 1e-5 to 1e-2
    }


    # Inner CV
    cv_inner = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

    # PR-AUC scorer
    def pr_auc_score(y_true, y_pred_proba):
        precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
        return auc(recall, precision)

    pr_auc_scorer = make_scorer(pr_auc_score, needs_proba=True)

    # Randomized Search
    random_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_grid_svc,
        scoring=pr_auc_scorer,
        cv=cv_inner,
        n_iter=50,
        n_jobs=-1,
        random_state=42
    )

    random_search.fit(X_train_outer, y_train_outer)

    best_model = random_search.best_estimator_
    best_params = random_search.best_params_

    best_model_dict[f"Outer {outer_rand}"] = best_model
    best_params_dict[f"Outer {outer_rand}"] = best_params

    # Predictions and evaluation
    y_pred = best_model.predict(X_test_outer)
    y_pred_proba = best_model.predict_proba(X_test_outer)[:, 1]

    tn, fp, fn, tp = confusion_matrix(y_test_outer, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    auc_score = roc_auc_score(y_test_outer, y_pred_proba)
    f1 = f1_score(y_test_outer, y_pred)
    accuracy = accuracy_score(y_test_outer, y_pred)

    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_proba)
    pr_auc_val = auc(recall, precision)

    precision_list.append(precision)
    recall_list.append(recall)
    auc_list.append(pr_auc_val)

    best_params_dict[f"Outer {outer_rand}"]['auc'] = auc_score
    best_params_dict[f"Outer {outer_rand}"]['f1'] = f1
    best_params_dict[f"Outer {outer_rand}"]['sensitivity'] = sensitivity
    best_params_dict[f"Outer {outer_rand}"]['specificity'] = specificity
    best_params_dict[f"Outer {outer_rand}"]['accuracy'] = accuracy
    best_params_dict[f"Outer {outer_rand}"]['pr_auc'] = pr_auc_val

    print(f"\nOuter {outer_rand} -> Best Hyperparameters: {best_params}")

    # Save results after each iteration
    results_df = pd.DataFrame.from_dict(best_params_dict, orient='index')
    results_csv_path = os.path.join(os.getcwd(), 'svm_best_hyperparameters_randomsearch.csv')
    results_df.to_csv(results_csv_path)
    print(f"\nBest parameters saved to {results_csv_path} after Outer {outer_rand} completion")

    # Save PR curve data
    pr_data = {
        "precision_list": precision_list,
        "recall_list": recall_list,
        "auc_list": auc_list
    }
    pr_data_path = os.path.join(os.getcwd(), 'svm_precision_recall_data_randomsearch.pkl')
    with open(pr_data_path, 'wb') as f:
        pickle.dump(pr_data, f)
    print(f"\nPrecision-recall data saved to {pr_data_path} after Outer {outer_rand} completion")


# Plot averaged PR curve
with open(pr_data_path, 'rb') as f:
    pr_data = pickle.load(f)

precision_list = pr_data["precision_list"]
recall_list = pr_data["recall_list"]
auc_list = pr_data["auc_list"]

mean_recall = np.linspace(0, 1, 100)
interp_precisions = []

for precision, recall in zip(precision_list, recall_list):
    order = np.argsort(recall)
    recall_sorted = recall[order]
    precision_sorted = precision[order]
    interp = np.interp(mean_recall, recall_sorted, precision_sorted, left=1.0, right=0.0)
    interp_precisions.append(interp)

mean_precision = np.mean(interp_precisions, axis=0)
std_precision = np.std(interp_precisions, axis=0)

plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, label=f"Mean PR Curve (AUC = {np.mean(auc_list):.2f})", color='purple')
plt.fill_between(mean_recall, mean_precision - std_precision, mean_precision + std_precision,
                 alpha=0.2, color='violet', label="±1 Std. Dev.")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("SVM Precision-Recall Curve (Mean ± Std) - RandomSearchCV")
plt.legend(loc="lower left")
plt.grid()
plt.tight_layout()

plot_path = os.path.join(os.getcwd(), 'svm_pr_curve_randomsearch.png')
plt.savefig(plot_path)
plt.show()


fatal: destination path 'tm10007_ml' already exists and is not an empty directory.


Outer Loop:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
Outer Loop:  10%|█         | 1/10 [01:55<17:19, 115.45s/it]


Outer 0 -> Best Hyperparameters: {'svc__C': np.float64(0.31489116479568624), 'svc__class_weight': 'balanced', 'svc__gamma': np.float64(0.000541675458324746), 'svc__shrinking': False, 'svc__tol': np.float64(0.0006986584841970366), 'auc': np.float64(0.3324943367732192), 'f1': 0.29743589743589743, 'sensitivity': np.float64(1.0), 'specificity': np.float64(0.0), 'accuracy': 0.1746987951807229, 'pr_auc': np.float64(0.1306144421906849)}

Best parameters saved to /content/svm_best_hyperparameters_randomsearch.csv after Outer 0 completion

Precision-recall data saved to /content/svm_precision_recall_data_randomsearch.pkl after Outer 0 completion


Outer Loop:  10%|█         | 1/10 [02:46<25:02, 166.98s/it]


KeyboardInterrupt: 